In [1]:
import geopandas as gpd
import datetime
import glob
import pandas as pd 
import rasterio
import numpy as np

import shapely
import altair as alt
sys.path.append("..")
import profiling_tools

In [2]:
xsection_file_dict = {
    "Deming": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/valley_xsections_from_glaciers.geojson",
    "Rainbow": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/valley_xsections_from_glaciers.geojson",
    "Thunder": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/whole_mountain/thunder_xsections.geojson",
    "Park": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/whole_mountain/park_xsections.geojson",
    "Squak": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/whole_mountain/squak_xsections.geojson",
    "Talum": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/whole_mountain/talum_xsections.geojson",
    "Boulder": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/whole_mountain/boulder_xsections.geojson",
    "Easton": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/easton/valley_xsections_from_glaciers.geojson",
    "Coleman": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/coleman/valley_xsections_from_glaciers.geojson",
    "Mazama": "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/mazama/valley_xsections_from_glaciers.geojson"
}
dem_files = [
    "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/whole_mountain/dems/2015_09_01.tif",
    "/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/whole_mountain/dems/1947_09_14.tif",
]
dem_timestamps = [
    datetime.datetime(2015, 9, 1),
    datetime.datetime(1947, 9, 14)
]

glacier_files = glob.glob("/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/**/glaciers.geojson", recursive=True)
strip_time_format = "%Y_%m_%d"

LINE_COMPLEXITY = 150

In [3]:
combined_gdf = gpd.GeoDataFrame()

for key, filename in xsection_file_dict.items():
    gdf = gpd.read_file(filename)
    gdf['Valley Name'] = key
    gdf.geometry = gdf.geometry.apply(lambda g: profiling_tools.increase_line_complexity(g, LINE_COMPLEXITY))
    gdf['centroid'] = gdf.geometry.apply(lambda x: x.centroid)
    gdf['coords'] = gdf.geometry.apply(lambda x: list(x.coords))
    gdf = gdf.explode('coords', ignore_index=True)
    gdf['coords'] = gdf['coords'].apply(shapely.geometry.Point)
    gdf.drop(columns=["geometry"])
    
    for dem_file,date in zip(dem_files, dem_timestamps):
        with rasterio.open(dem_file) as src:
            new_gdf = gdf.copy()
            new_gdf['elevation'] = pd.Series(list(src.sample(new_gdf["coords"].apply(lambda x: list(x.coords)[0]))))
            new_gdf['elevation'] = new_gdf['elevation'].apply(lambda x: np.nan if x == src.nodata else x[0])

            # Find the point in each cross section line (identified by the ID column, with 0 meaning furthest downstream) with the lowest elevation
            new_gdf = new_gdf.sort_values('elevation').groupby('n_from_glacial_max').apply(pd.DataFrame.head, n=1)
            new_gdf['low_point_coords'] = new_gdf.apply(lambda row: None if np.isnan(row['elevation']) else row['coords'], axis=1)

        # Set the geometry to t     est downstream cross-section
            new_gdf.geometry = new_gdf["centroid"]
            new_gdf['path_distance'] = pd.Series(new_gdf.distance(
                    gpd.GeoDataFrame(new_gdf.shift(1), crs=new_gdf.crs)
                ).fillna(0)).cumsum()
            new_gdf['date'] = date
            combined_gdf = combined_gdf.append(new_gdf)

def create_path_dist_from_glaciers(df):
    path_distance_at_glacier = df.loc[df['n_from_glacial_max']==0, 'path_distance'].iloc[0]
    df['path_distance_from_glacier'] = path_distance_at_glacier - df['path_distance']
    return df
combined_gdf = combined_gdf.groupby('date').apply(create_path_dist_from_glaciers)
combined_gdf['path_distance_from_glacier'] = - combined_gdf['path_distance_from_glacier']
combined_gdf = combined_gdf.set_crs(crs=gdf.crs)
combined_gdf  = combined_gdf.reset_index(drop=True)

# Mark points as (non)glacial

In [4]:
glaciers_gdf = pd.concat([gpd.read_file(f) for f in glacier_files])
glaciers_gdf = glaciers_gdf.to_crs(combined_gdf.crs)
glaciers_gdf = glaciers_gdf[glaciers_gdf['year'].notna()].query("year != ''")
glaciers_gdf['time'] = glaciers_gdf['year'].apply(lambda d: datetime.datetime.strptime(d, strip_time_format))


In [5]:
combined_gdf['glacial'] = combined_gdf.apply(
    lambda row: any(glaciers_gdf.loc[glaciers_gdf['time'] == row["date"], 'geometry'].apply(lambda g: g.contains(row['coords']))),
    axis=1
)

In [6]:
combined_w_geoms_gdf = combined_gdf.copy(deep=True)
combined_gdf = combined_w_geoms_gdf.drop(columns=['geometry', 'centroid', 'coords', 'low_point_coords'])

In [7]:
combined_gdf['year'] = combined_gdf['date'].dt.year

In [8]:
alt.Chart(
    combined_gdf
).mark_line(thickness=10).encode(
    alt.X('path_distance_from_glacier'),
    alt.Y('elevation', scale = alt.Scale(zero=False)),
    alt.Facet('Valley Name'),
    alt.Color("year:N"),
    alt.StrokeDash("glacial")
).resolve_scale(
    x ='independent',
    y='independent'
)

alt.Chart(...)

# Calculate slope using the uppermost and lower most elevation measurement

In [9]:
ends_df = combined_gdf
ends_df = ends_df.groupby(["Valley Name", "year"]).apply(lambda df:
    df[
        (df['n_from_glacial_max'] == 0) | (df['n_from_glacial_max'] == df['n_from_glacial_max'].max())
    ]
).reset_index(drop=True)

slopes_df = ends_df.copy()
slopes_df[['path_distance_from_glacier', 'elevation']] = slopes_df.groupby(["Valley Name", "year"])[['path_distance_from_glacier', 'elevation']].diff()
slopes_df['slope'] = np.degrees(np.arctan(
    slopes_df['elevation']/slopes_df['path_distance_from_glacier']
))
slopes_df = slopes_df.reset_index()
slopes_df['slope'] = - slopes_df['slope']

In [10]:
alt.Chart(slopes_df).mark_bar().encode(
    alt.X("Valley Name"),
    alt.Y("slope")
)

alt.Chart(...)

# Calculate slope using measurements between glacial maximum and LIA maximum

Rainbow: 0 - 72

Mazama: 0 - 63

Deming: 0 - 69

All else, all xsections are within the LIA max, if we know where that is

In [11]:
combined_gdf_within_lia = combined_w_geoms_gdf.copy(deep=True).drop(columns=['geometry', 'centroid', 'coords', 'low_point_coords'])
combined_gdf_within_lia = pd.concat([
    combined_gdf[combined_gdf["Valley Name"] == 'Rainbow'].query("n_from_glacial_max > -1").query("n_from_glacial_max < 73"),
    combined_gdf[combined_gdf["Valley Name"] == 'Mazama'].query("n_from_glacial_max > -1").query("n_from_glacial_max < 64"),
    combined_gdf[combined_gdf["Valley Name"] == 'Deming'].query("n_from_glacial_max > -1").query("n_from_glacial_max < 70"),
    combined_gdf[~combined_gdf["Valley Name"].isin(["Rainbow", "Mazama", "Deming"])]
])

In [12]:
ends_df_within_lia = combined_gdf_within_lia
ends_df_within_lia = ends_df_within_lia.groupby(["Valley Name", "year"]).apply(lambda df:
    df[
        (df['n_from_glacial_max'] == 0) | (df['n_from_glacial_max'] == df['n_from_glacial_max'].max())
    ]
).reset_index(drop=True)

slopes_df_within_lia = ends_df_within_lia.copy()
slopes_df_within_lia[['path_distance_from_glacier', 'elevation']] = slopes_df_within_lia.groupby(["Valley Name", "year"])[['path_distance_from_glacier', 'elevation']].diff()
slopes_df_within_lia['slope'] = np.degrees(np.arctan(
    slopes_df_within_lia['elevation']/slopes_df_within_lia['path_distance_from_glacier']
))
slopes_df_within_lia = slopes_df_within_lia.reset_index()
slopes_df_within_lia['slope'] = - slopes_df_within_lia['slope']

In [13]:
slopes_df_within_lia['measurement'] = 'Between observed glacial max and LIA'
slopes_df['measurement'] = 'All measurable area'

slopes_df = pd.concat([
    slopes_df_within_lia,
    slopes_df
])

In [14]:
ls outputs | grep ".csv"

lithology.csv
modeling_powerlaw_data.csv
power_law_data_new_6points.csv
power_law_data_new.csv
slopes.csv
terrain_attributes_erosionarea_bytype.csv
terrain_attributes_erosionarea.csv
terrain_attributes_processpolygons.csv
terrain_attributes_processpolygons_maxdrainage.csv
terrain_attributes_watershedarea.csv
uncertainty.csv


In [15]:
slopes_df = slopes_df[[
    'Valley Name', 'year', 'slope','measurement'
]].dropna()

In [16]:
slopes_df.to_csv("outputs/slopes.csv")

In [17]:
slopes_df

,Valley Name,year,slope,measurement
1,Boulder,1947,10.760857,Between observed glacial max and LIA
3,Boulder,2015,10.789069,Between observed glacial max and LIA
5,Coleman,1947,7.138006,Between observed glacial max and LIA
7,Coleman,2015,7.216633,Between observed glacial max and LIA
9,Deming,1947,6.586342,Between observed glacial max and LIA
11,Deming,2015,6.531251,Between observed glacial max and LIA
13,Easton,1947,9.685916,Between observed glacial max and LIA
15,Easton,2015,9.818451,Between observed glacial max and LIA
17,Mazama,1947,8.421598,Between observed glacial max and LIA
19,Mazama,2015,8.447960,Between observed glacial max and LIA
